In [1]:
using CSV
using Dates
using DataFrames

In [3]:
ENV["COLUMNS"] = 1000;

In [30]:
forecast_rawdata = DataFrame(CSV.File("../rawdata/2021-02-01-all-forecasted-cases-model-data.csv"))
filter!(row -> row.model == "Ensemble", forecast_rawdata)
filter!(row -> row.location_name != row.State, forecast_rawdata)
@show nrow(forecast_rawdata)
first(forecast_rawdata, 5)

nrow(forecast_rawdata) = 12564


,model,forecast_date,target,target_end_date,location_name,State,fips,point,quantile_0.025,quantile_0.25,quantile_0.75,quantile_0.975
,String,Date,String,Date,String,String,String,Int64,Int64,Int64,Int64,Int64
1,Ensemble,2021-02-01,1 wk ahead inc case,2021-02-06,"Autauga County, Alabama",Alabama,01001,223,82,146,265,354
2,Ensemble,2021-02-01,1 wk ahead inc case,2021-02-06,"Baldwin County, Alabama",Alabama,01003,814,477,670,916,1089
3,Ensemble,2021-02-01,1 wk ahead inc case,2021-02-06,"Barbour County, Alabama",Alabama,01005,108,49,88,128,166
4,Ensemble,2021-02-01,1 wk ahead inc case,2021-02-06,"Bibb County, Alabama",Alabama,01007,53,18,39,79,103
5,Ensemble,2021-02-01,1 wk ahead inc case,2021-02-06,"Blount County, Alabama",Alabama,01009,184,83,142,207,271


In [32]:
hhs_rawdata = DataFrame(CSV.File("../data/hhs_data_2021_01_24.csv"))
@show nrow(hhs_rawdata)
first(hhs_rawdata, 5)

nrow(hhs_rawdata) = 824600


,hospital,hospital_id,date,admissions_icu,admissions_acute,admissions_allbeds,active_icu,active_acute,active_allbeds
,String,String,Date,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2020-07-31,0.471429,1.1,1.57143,1.71429,1.42857,3.14286
2,ABBEVILLE AREA MEDICAL CENTER,421301,2020-08-01,0.446939,1.04286,1.4898,1.59184,1.73469,3.32653
3,ABBEVILLE AREA MEDICAL CENTER,421301,2020-08-02,0.422449,0.985714,1.40816,1.46939,2.04082,3.5102
4,ABBEVILLE AREA MEDICAL CENTER,421301,2020-08-03,0.397959,0.928571,1.32653,1.34694,2.34694,3.69388
5,ABBEVILLE AREA MEDICAL CENTER,421301,2020-08-04,0.373469,0.871429,1.2449,1.22449,2.65306,3.87755


In [12]:
hospital_metadata = DataFrame(CSV.File("../data/hhs_hospital_meta.csv"))
@show nrow(hospital_metadata)
first(hospital_metadata, 5)

nrow(hospital_metadata) = 4928


,hospital,hospitalname,hospital_id,state,state_abbrev,city,zip,fips_code,system_id,system_name,hsa_id,hsa_name,hrr_id,hrr_name
,String,String,String,String,String,String?,Int64?,Int64?,String?,String?,Int64?,String?,Int64?,String?
1,ABBEVILLE AREA MEDICAL CENTER,Abbeville Area Medical Center,421301,South Carolina,SC,Abbeville,29620,45001,missing,missing,42001,"Abbeville, SC",366,"Columbia, SC"
2,ABBEVILLE GENERAL HOSPITAL,Abbeville General Hospital,190034,Louisiana,LA,Abbeville,70510,22113,HSI00000562,Lafayette General Health,19001,"Abbeville, LA",213,"Lafayette, LA"
3,ABBOTT NORTHWESTERN HOSPITAL,Abbott Northwestern Hospital,240057,Minnesota,MN,Minneapolis,55407,27053,HSI00000029,Allina Health System,24076,"Minneapolis, MN",251,"Minneapolis, MN"
4,ABILENE REGIONAL MEDICAL CENTER,Abilene Regional Medical Center,450558,Texas,TX,Abilene,79606,48441,HSI00000249,Community Health Systems,45001,"Abilene, TX",382,"Abilene, TX"
5,ABINGTON MEMORIAL HOSPITAL,Abington Memorial Hospital,390231,Pennsylvania,PA,Abington,19001,42091,HSI00000048,Jefferson Health,39001,"Abington, PA",356,"Philadelphia, PA"


In [23]:
zipcountydata = DataFrame(CSV.File("../rawdata/zip_to_county.csv"))
zip_to_fips = Dict(row.zip => row.stcountyfp for row in eachrow(zipcountydata));

In [25]:
hospital_metadata.fips_code = map(row -> (ismissing(row.fips_code) && !ismissing(row.zip)) ? zip_to_fips[row.zip] : row.fips_code, eachrow(hospital_metadata));

In [33]:
hospitals_by_fips = Dict(f => String[] for f in skipmissing(unique(hospital_metadata.fips_code)));
for h in eachrow(hospital_metadata)
    if ismissing(h.fips_code) continue end
    push!(hospitals_by_fips[h.fips_code], h.hospital_id)
end

In [35]:
hospitals_by_fips

Dict{Int64,Array{String,1}} with 2465 entries:
  30027 => ["271345"]
  9011  => ["070007", "070024"]
  44005 => ["410006"]
  16079 => ["131314"]
  5055  => ["040039"]
  17179 => ["141330", "140120"]
  22035 => ["190208"]
  16059 => ["131305"]
  39001 => ["361326"]
  46009 => ["431317", "431327"]
  13019 => ["110234"]
  28161 => ["250061"]
  13047 => ["110236"]
  17131 => ["141304"]
  1131  => ["010102"]
  47011 => ["440185"]
  19187 => ["160016"]
  46089 => ["431308"]
  48087 => ["451355"]
  19013 => ["160110", "160040", "160067"]
  13077 => ["110229", "110233"]
  13161 => ["111333"]
  29147 => ["260050"]
  48303 => ["453306", "45C0001441", "450040", "452102", "450162", "450876", "450686"]
  40097 => ["370015"]
  ⋮     => ⋮

In [39]:
hsaf = DataFrame(CSV.File("../rawdata/HSAF_2019.csv"));
rename!(hsaf,
    :MEDICARE_PROV_NUM => :hospital_id,
    :ZIP_CD_OF_RESIDENCE => :zipcode,
    :TOTAL_DAYS_OF_CARE => :total_daysofcare,
    :TOTAL_CHARGES => :total_charges,
    :TOTAL_CASES => :total_patients,
);

In [40]:
first(hsaf, 5)

,hospital_id,zipcode,total_daysofcare,total_charges,total_patients
,String,Int64?,Int64,Int64?,Int64?
1,010001,1913,2,missing,missing
2,010001,3811,8,missing,missing
3,010001,7206,6,missing,missing
4,010001,8066,2,missing,missing
5,010001,8234,4,missing,missing


In [43]:
hsaf_hospitalids = sort(unique(hsaf.hospital_id));
hhs_hospitalids = sort(unique(hospital_metadata.hospital_id));

In [44]:
setdiff(hsaf_hospitalids, hhs_hospitalids)

1175-element Array{String,1}:
 "010018"
 "010032"
 "010047"
 "010109"
 "013025"
 "013028"
 "013029"
 "013030"
 "013031"
 "013032"
 "013033"
 "014000"
 "014006"
 ⋮
 "673064"
 "673065"
 "673066"
 "673067"
 "673068"
 "673069"
 "673070"
 "673071"
 "673072"
 "673073"
 "673074"
 "713025"